In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import file_utils 
import db_utils
import config
#set working directory
os.chdir(config.working_dir)

In [5]:
pd.set_option('display.max_columns', 500)

In [6]:
# db credentials authentication

In [7]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [8]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [9]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_data = pd.read_sql_query('select * from cs_retail_loan_cust', con)
con.close()

In [10]:
# get inflow/ouflow for retail loan customers only and insert into database

for i, f in enumerate(file_utils.get_files(r"N:\10. Executive\Original\ETL\in_out_flow")):
    print i+1, '. ', f
    cur_df = file_utils.unzip(f)
    cur_df = cs_retail_loan_cust_data[['BRA_CODE','CUS_NUM']].merge(
        cur_df, on=['BRA_CODE','CUS_NUM'], how='inner').fillna(0)
        
    if_exists_var = 'append'
    if i==0:
        if_exists_var = 'replace'
    
    cur_df['REFERENCE_DATE'] = pd.to_datetime(cur_df['REFERENCE_DATE'], format="%d%b%Y")
    cur_df['CTRL_DATE'] = pd.to_datetime(cur_df['CTRL_DATE'], format="%d%b%Y")
    
    conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))    
    cur_df.to_sql('cs_retail_loans_in_out_flow', conn, index=False, if_exists=if_exists_var, chunksize=10000,
                                                                      dtype=db_utils.sqlcol(cur_df))
    

1 .  N:\10. Executive\Original\ETL\in_out_flow\17Feb2020.csv.gz
2 .  N:\10. Executive\Original\ETL\in_out_flow\28Feb2018.csv.gz
3 .  N:\10. Executive\Original\ETL\in_out_flow\28Feb2019.csv.gz
4 .  N:\10. Executive\Original\ETL\in_out_flow\30Apr2018.csv.gz
5 .  N:\10. Executive\Original\ETL\in_out_flow\30Apr2019.csv.gz
6 .  N:\10. Executive\Original\ETL\in_out_flow\30Jun2018.csv.gz
7 .  N:\10. Executive\Original\ETL\in_out_flow\30Jun2019.csv.gz
8 .  N:\10. Executive\Original\ETL\in_out_flow\30Nov2018.csv.gz
9 .  N:\10. Executive\Original\ETL\in_out_flow\30Nov2019.csv.gz
10 .  N:\10. Executive\Original\ETL\in_out_flow\30Sep2018.csv.gz
11 .  N:\10. Executive\Original\ETL\in_out_flow\30Sep2019.csv.gz
12 .  N:\10. Executive\Original\ETL\in_out_flow\31Aug2018.csv.gz
13 .  N:\10. Executive\Original\ETL\in_out_flow\31Aug2019.csv.gz
14 .  N:\10. Executive\Original\ETL\in_out_flow\31Dec2018.csv.gz
15 .  N:\10. Executive\Original\ETL\in_out_flow\31Dec2019.csv.gz
16 .  N:\10. Executive\Original\ET

#### Summarize: Turnover/Lodgement for Retail Loan Customers, derive stanadard deviation and mean

In [11]:
print 'get inflow/outflow (12months) of retail loan customers...'

con = cx_Oracle.connect(conn_string)
df_retail_loan_cust_turn_lodge_count = pd.read_sql_query(
'''
select bra_code, cus_num,
sum(lodgement_amount) sum_lodgement_amount,
sum(lodgement_count) sum_lodgement_count,
sum(turnover_amount) sum_turnover_amount,
sum(turnover_count) sum_turnover_count,
sqrt(sum(lodge_amt_std_compo)/avg(std_dev_n)) std_dev_lodgement_amount,
sqrt(sum(lodge_cnt_std_compo)/avg(std_dev_n)) std_dev_lodgement_count,
sqrt(sum(turn_amt_std_compo)/avg(std_dev_n)) std_dev_turnover_amount,
sqrt(sum(turn_cnt_std_compo)/avg(std_dev_n)) std_dev_turnover_count,
count(1) as months_active_in_12_mths,
avg(case when months_with_gtb>12 then 12 else months_with_gtb end) months_with_gtb_in_12_mths
from
(select a.bra_code, a.cus_num, a.is_bad, aa.date_open cus_date_open,
months_between(trunc(a.date_open,'month'), trunc(aa.date_open,'month')) months_with_gtb,
b.reference_date,a.date_open,
lodgement_amount, turnover_amount, lodgement_count, turnover_count,
(case when months_between(a.date_open, aa.date_open)>12 then 12 else round(months_between(a.date_open, aa.date_open)) end) as std_dev_n,
power(lodgement_amount - avg(lodgement_amount) over(partition by a.bra_code, a.cus_num), 2) lodge_amt_std_compo,
power(lodgement_count - avg(lodgement_count) over(partition by a.bra_code, a.cus_num), 2) lodge_cnt_std_compo,
power(turnover_amount - avg(turnover_amount) over(partition by a.bra_code, a.cus_num), 2) turn_amt_std_compo,
power(turnover_count - avg(turnover_count) over(partition by a.bra_code, a.cus_num), 2) turn_cnt_std_compo
from
cs_retail_loan_cust a
inner join stg.src_customer aa
on a.bra_code=aa.bra_code and a.cus_num=aa.cus_num
inner join cs_retail_loans_in_out_flow b
on a.bra_code=b.bra_code and a.cus_num=b.cus_num
and b.reference_date between trunc(add_months(a.date_open, -12),'month') and trunc(add_months(a.date_open, -1),'month')
)
group by bra_code, cus_num''', con)
con.close()

get inflow/outflow (12months) of retail loan customers...


In [12]:
# further computation

In [13]:
df_retail_loan_cust_turn_lodge_count['AVG_LODGEMENT_AMOUNT'] = \
df_retail_loan_cust_turn_lodge_count['SUM_LODGEMENT_AMOUNT']/df_retail_loan_cust_turn_lodge_count['MONTHS_WITH_GTB_IN_12_MTHS']

df_retail_loan_cust_turn_lodge_count['AVG_LODGEMENT_COUNT'] = \
df_retail_loan_cust_turn_lodge_count['SUM_LODGEMENT_COUNT']/df_retail_loan_cust_turn_lodge_count['MONTHS_WITH_GTB_IN_12_MTHS']

df_retail_loan_cust_turn_lodge_count['AVG_TURNOVER_AMOUNT'] = \
df_retail_loan_cust_turn_lodge_count['SUM_TURNOVER_AMOUNT']/df_retail_loan_cust_turn_lodge_count['MONTHS_WITH_GTB_IN_12_MTHS']

df_retail_loan_cust_turn_lodge_count['AVG_TURNOVER_COUNT'] = \
df_retail_loan_cust_turn_lodge_count['SUM_TURNOVER_COUNT']/df_retail_loan_cust_turn_lodge_count['MONTHS_WITH_GTB_IN_12_MTHS']


In [14]:
df_retail_loan_cust_turn_lodge_count['IN_OUT_AMT_RATIO'] = \
df_retail_loan_cust_turn_lodge_count['SUM_LODGEMENT_AMOUNT']/df_retail_loan_cust_turn_lodge_count['SUM_TURNOVER_AMOUNT']

df_retail_loan_cust_turn_lodge_count['IN_OUT_CNT_RATIO'] = \
df_retail_loan_cust_turn_lodge_count['SUM_LODGEMENT_COUNT']/df_retail_loan_cust_turn_lodge_count['SUM_TURNOVER_COUNT']

df_retail_loan_cust_turn_lodge_count['COV_TURNOVER_AMT']= \
df_retail_loan_cust_turn_lodge_count['STD_DEV_TURNOVER_AMOUNT']/df_retail_loan_cust_turn_lodge_count['AVG_TURNOVER_AMOUNT']

df_retail_loan_cust_turn_lodge_count['COV_LODGEMENT_AMT']=\
df_retail_loan_cust_turn_lodge_count['STD_DEV_LODGEMENT_AMOUNT']/df_retail_loan_cust_turn_lodge_count['AVG_LODGEMENT_AMOUNT']

df_retail_loan_cust_turn_lodge_count['COV_TURNOVER_CNT']=\
df_retail_loan_cust_turn_lodge_count['STD_DEV_TURNOVER_COUNT']/df_retail_loan_cust_turn_lodge_count['AVG_TURNOVER_COUNT']

df_retail_loan_cust_turn_lodge_count['COV_LODGEMENT_CNT']=\
df_retail_loan_cust_turn_lodge_count['STD_DEV_LODGEMENT_COUNT']/df_retail_loan_cust_turn_lodge_count['AVG_LODGEMENT_COUNT']

df_retail_loan_cust_turn_lodge_count['MONTH_ACTIVE_RATIO']=\
df_retail_loan_cust_turn_lodge_count['MONTHS_ACTIVE_IN_12_MTHS']/df_retail_loan_cust_turn_lodge_count['MONTHS_WITH_GTB_IN_12_MTHS']

In [15]:
# df_retail_loan_cust_turn_lodge_count[df_retail_loan_cust_turn_lodge_count['MONTH_ACTIVE_RATIO']<0.9].IS_BAD.value_counts()

In [16]:
df_retail_loan_cust_turn_lodge_count.replace([np.inf,np.nan], 0, inplace=True)

In [17]:
df_retail_loan_cust_turn_lodge_count.to_csv('cs_retail_loan_cust_turn_lodge_count_info.csv', index=False)